In [ ]:
import threading

# List to store client objects and threads
clients = []
threads = []

# Configure different clients - all using the same port for broadcast reception
client_configs = [
    {"client_id": "client_1", "client_port": 37020, "max_runtime": 300},
    {"client_id": "client_2", "client_port": 37020, "max_runtime": 300},
    {"client_id": "client_3", "client_port": 37020, "max_runtime": 300}
]

# Create and start each client in its own thread
for config in client_configs:
    # Create client instance
    client = UDP_Client(**config)
    clients.append(client)
    
    # Create and start thread
    thread = threading.Thread(target=client.run_client)
    thread.daemon = True  # Thread will exit when main program exits
    thread.start()
    threads.append(thread)

print(f"Started {len(clients)} clients in separate threads")

In [ ]:
import threading

# List to store client objects and threads
clients = []
threads = []
handlers = []  # List to store handler instances

# Configure different clients - all using the same port for broadcast reception
client_configs = [
    {"client_id": "client_1", "client_port": 37020, "max_runtime": 300},
    {"client_id": "client_2", "client_port": 37020, "max_runtime": 300},
    {"client_id": "client_3", "client_port": 37020, "max_runtime": 300}
]

# Create handlers for each client (assuming SerialPoller class)
# Each with different serial ports or configurations
handler_configs = [
    {"serial_port": "COM3", "baudrate": 9600, "poll_interval": 1.0},
    {"serial_port": "COM4", "baudrate": 115200, "poll_interval": 0.5},
    {"serial_port": "COM5", "baudrate": 9600, "poll_interval": 2.0}
]

# Create and start each client in its own thread
for i, (client_config, handler_config) in enumerate(zip(client_configs, handler_configs)):
    # Create client instance
    client = UDP_Client(**client_config)
    clients.append(client)
    
    # Create handler instance
    handler = SerialPoller(**handler_config)
    handlers.append(handler)
    
    # Create thread with both client and its handler
    # We'll use a lambda to call run_client with the handler.handle_events function
    thread = threading.Thread(
        target=lambda c=client, h=handler: c.run_client(between_events_func=h.handle_events)
    )
    thread.daemon = True  # Thread will exit when main program exits
    thread.start()
    threads.append(thread)

print(f"Started {len(clients)} clients in separate threads, each with a unique handler")

# Make sure to clean up handlers when done
def cleanup():
    for handler in handlers:
        try:
            handler.cleanup()
        except Exception as e:
            print(f"Error cleaning up handler: {e}")

# Use a try-finally block to ensure cleanup happens
try:
    # Wait for all threads to complete (or use other control mechanism)
    for thread in threads:
        thread.join()
except KeyboardInterrupt:
    print("Interrupted by user")
finally:
    cleanup()